In [86]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data
    
def create_data_struct():
    modules = {}
    types = {}
    for line in get_data(20):
        key, val = line.split(' -> ')
        if key == 'broadcaster':
            type = 'b'
        else:
            type = key[0]
            key = key[1:]
        modules[key] = val.split(', ')
        types[key] = type
    return modules, types
        

def find_parent_state(child_state, modules, types):
    out = []
    for k,v in modules.items():
        if child_state in v:
            out.append((k, types[k]))
    return out
    


from collections import deque, defaultdict
from math import gcd

def lcm(xs):
  ans = 1
  for x in xs:
    ans = (ans*x)//gcd(x,ans)
  return ans

In [ ]:
modules, types = create_data_struct()


connector_memory = {}
INV = defaultdict(list)

for from_module, to_module_list in modules.items():
    for to_module in to_module_list:
        if (to_module in types) and (types[to_module]=='&'):
            if to_module not in connector_memory:
                connector_memory[to_module] = {}
            connector_memory[to_module][from_module] = False
        INV[to_module].append(from_module)


FROM = {}
INV = defaultdict(list)
for x,ys in R.items():
  R[x] = [adjust(y) for y in ys]
  for y in R[x]:
    if y[0]=='&':
      if y not in FROM:
        FROM[y] = {}
      FROM[y][x] = 'lo'
    INV[y].append(x)

assert len(INV['rx'])==1
assert INV['rx'][0][0]=='&'
WATCH = INV[INV['rx'][0]]

Q = deque()
PREV = {}
COUNT = defaultdict(int)
TO_LCM = []
ON = set()
hi = 0
lo = 0

for ii in range(10**10):
    state = ('broadcaster', 'button', False) # to from pulse
    Q.append(state)

    while Q:
        to_module, from_module, pulse  = Q.popleft()


        ### part2 hack
        if to_module in PREV and COUNT[to_module]==2 and to_module in WATCH:
            TO_LCM.append(ii-PREV[to_module])
        PREV[to_module] = t
        COUNT[to_module] += 1
        if len(TO_LCM) == len(WATCH):
            print(lcm(TO_LCM))
            sys.exit(0)
            
        if pulse:
            hi += 1
        else:
            lo += 1

        if to_module not in modules:
            continue
       
        if to_module=='broadcaster':
            for next_module in modules[to_module]:
                Q.append((next_module, to_module, pulse))
        
        elif types[to_module]=='%':
            if pulse:
                continue
            else:
                if to_module not in ON:
                    ON.add(to_module)
                    pulse = True
                else:
                    ON.discard(to_module)
                    pulse = False
                for next_module in modules[to_module]:
                    Q.append((next_module, to_module, pulse))
                    
        elif types[to_module] == '&':
            connector_memory[to_module][from_module] = pulse
            new_pulse  = (False if all(y==True for y in connector_memory[to_module].values()) else True)
            for next_module in modules[to_module]:
                Q.append((next_module, to_module, new_pulse))
        else:
             assert False, to_module
    if ii == 999:
        print(lo*hi)



711650489
